# Positive Negative Review Predictions

# Installing Libraries

In [1]:
sc.install_pypi_package("pandas==0.25.1")
sc.install_pypi_package("textblob")
sc.install_pypi_package("matplotlib==3.1.1", "https://pypi.org/simple")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1649754617506_0008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…




You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

# Importing Libraries

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import * 
from pyspark.ml.tuning import CrossValidator
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import string 
import re 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Data Loading through pipeline

In [3]:
train = spark.read.parquet("hdfs:///pipeline_train_o/p/").select("text","featuresUni")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
test= spark.read.parquet("hdfs:///pipeline_test_o/p/").select("text","featuresUni")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
train.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|                text|         featuresUni|
+--------------------+--------------------+
|Ordered a cheese ...|(262144,[2437,719...|
|Got a pedicure th...|(262144,[5325,967...|
+--------------------+--------------------+
only showing top 2 rows

# Function to get sentiment

In [6]:
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]   
    if temp > 0.0:
        return 2.0 # Positive
    elif temp == 0.0:
        return 1.0 # Neutral
    else:
        return 0.0 # Negative
# UDF to write sentiment on DF
sentiment = udf(apply_blob, DoubleType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
train=train.withColumn("label", sentiment(train['text']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
test=test.withColumn("label", sentiment(test['text']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Display data with labeled

In [9]:
train.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|                text|         featuresUni|label|
+--------------------+--------------------+-----+
|Ordered a cheese ...|(262144,[2437,719...|  0.0|
|Got a pedicure th...|(262144,[5325,967...|  2.0|
|I know, it's tour...|(262144,[5325,690...|  2.0|
|I go to this Chip...|(262144,[10879,13...|  2.0|
|Seating Process:
...|(262144,[991,3067...|  2.0|
|Excellent wings. ...|(262144,[3067,826...|  2.0|
|Came here once ma...|(262144,[2437,517...|  2.0|
|I used to love co...|(262144,[22567,61...|  0.0|
|I'm not quite sur...|(262144,[4399,106...|  2.0|
|Hot pot restauran...|(262144,[26623,42...|  2.0|
|Just got my septu...|(262144,[22567,97...|  2.0|
|I'm sorry, but th...|(262144,[2437,566...|  0.0|
|Have not been her...|(262144,[25615,70...|  2.0|
|This Starbucks de...|(262144,[7367,978...|  0.0|
|If I could go bac...|(262144,[2276,372...|  0.0|
|I really love IHO...|(262144,[22567,34...|  2.0|
|Walked away with ...|(262144,[27505,40...|  1.0|


In [10]:
train=train.withColumnRenamed("featuresUni","features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
test=train.withColumnRenamed("featuresUni","features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|                text|            features|label|
+--------------------+--------------------+-----+
|Ordered a cheese ...|(262144,[2437,719...|  0.0|
|Got a pedicure th...|(262144,[5325,967...|  2.0|
|I know, it's tour...|(262144,[5325,690...|  2.0|
|I go to this Chip...|(262144,[10879,13...|  2.0|
|Seating Process:
...|(262144,[991,3067...|  2.0|
|Excellent wings. ...|(262144,[3067,826...|  2.0|
|Came here once ma...|(262144,[2437,517...|  2.0|
|I used to love co...|(262144,[22567,61...|  0.0|
|I'm not quite sur...|(262144,[4399,106...|  2.0|
|Hot pot restauran...|(262144,[26623,42...|  2.0|
|Just got my septu...|(262144,[22567,97...|  2.0|
|I'm sorry, but th...|(262144,[2437,566...|  0.0|
|Have not been her...|(262144,[25615,70...|  2.0|
|This Starbucks de...|(262144,[7367,978...|  0.0|
|If I could go bac...|(262144,[2276,372...|  0.0|
|I really love IHO...|(262144,[22567,34...|  2.0|
|Walked away with ...|(262144,[27505,40...|  1.0|


In [13]:
train.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)

# LogisticRegression

In [14]:
from pyspark.ml.classification import LogisticRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
lr = LogisticRegression(maxIter=50, regParam=0.3, elasticNetParam=0,standardization=True, family="multinomial")
lrModel = lr.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
trainpredictions = lrModel.transform(train)
predictions = lrModel.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Model Evaluation

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.829918305977356

In [18]:
print("Training F1 with logistic regression",evaluator.evaluate(trainpredictions,{evaluator.metricName: "f1"}))
print("Training weightedRecall with logistic regression",evaluator.evaluate(trainpredictions,{evaluator.metricName: "weightedRecall"}))
print("Training weightedPrecision with logistic regression",evaluator.evaluate(trainpredictions,{evaluator.metricName: "weightedPrecision"}))
print("Training accuracy with logistic regression",evaluator.evaluate(trainpredictions,{evaluator.metricName: "accuracy"}))
print("Test accuracy with logistic regression",evaluator.evaluate(predictions,{evaluator.metricName: "accuracy"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training F1 with logistic regression 0.829918305977356
Training weightedRecall with logistic regression 0.8582662018277203
Training weightedPrecision with logistic regression 0.8617007491358581
Training accuracy with logistic regression 0.8582662018277203
Test accuracy with logistic regression 0.8582662018277203